# 🎬 ZERO STUDIO - Geração de Vídeos com IA

**Geração profissional de vídeos com Lip-Sync usando IA**

Este notebook permite executar o ZERO STUDIO diretamente no Google Colab.

---

## Funcionalidades:
- ✅ Geração de vídeos com lip-sync
- ✅ Suporte a múltiplos roteiros em lote
- ✅ Integração com ElevenLabs e MiniMax para áudio
- ✅ Integração com WaveSpeed para geração de vídeo
- ✅ Sistema de avatares personalizados
- ✅ Gerenciamento de projetos

---

## 📦 Passo 1: Instalar Dependências

Execute esta célula para instalar todas as bibliotecas necessárias.

In [ ]:
# Instalar dependências do projeto
!pip install -q flask flask-cors python-dotenv requests pillow google-generativeai pyngrok

print("✅ Dependências instaladas com sucesso!")

## 📥 Passo 2: Clonar o Repositório

Clone o repositório do ZERO STUDIO do GitHub.

In [ ]:
import os

# Remover pasta existente se houver
!rm -rf /content/zero-studio

# Clone o repositório (substitua pela URL do seu repositório)
!git clone https://github.com/sterling9879/oficial-1.git /content/zero-studio

# Entrar na pasta do projeto
os.chdir('/content/zero-studio')

# Criar pastas necessárias
!mkdir -p temp/uploads temp/outputs temp/audios temp/videos data/avatars/thumbnails

print("✅ Repositório clonado com sucesso!")
print(f"📁 Diretório atual: {os.getcwd()}")

## 🔑 Passo 3: Configurar API Keys

⚠️ **IMPORTANTE:** Preencha suas API keys abaixo antes de executar.

Você pode obter as chaves em:
- **ElevenLabs:** https://elevenlabs.io/
- **MiniMax:** https://www.minimax.chat/
- **Gemini:** https://aistudio.google.com/
- **WaveSpeed:** https://wavespeed.ai/

In [ ]:
#@title 🔐 Configure suas API Keys { display-mode: "form" }

#@markdown ### Preencha suas API Keys:

ELEVENLABS_API_KEY = "" #@param {type:"string"}
MINIMAX_API_KEY = "" #@param {type:"string"}
GEMINI_API_KEY = "" #@param {type:"string"}
WAVESPEED_API_KEY = "" #@param {type:"string"}

#@markdown ---
#@markdown ### Configuração do ngrok (opcional para URL personalizada):
NGROK_AUTH_TOKEN = "" #@param {type:"string"}

# Criar arquivo .env
env_content = f"""ELEVENLABS_API_KEY={ELEVENLABS_API_KEY}
MINIMAX_API_KEY={MINIMAX_API_KEY}
GEMINI_API_KEY={GEMINI_API_KEY}
WAVESPEED_API_KEY={WAVESPEED_API_KEY}
"""

with open('/content/zero-studio/.env', 'w') as f:
    f.write(env_content)

# Verificar quais keys foram configuradas
print("\n📋 Status das API Keys:")
print(f"   ElevenLabs: {'✅ Configurada' if ELEVENLABS_API_KEY else '❌ Não configurada'}")
print(f"   MiniMax:    {'✅ Configurada' if MINIMAX_API_KEY else '❌ Não configurada'}")
print(f"   Gemini:     {'✅ Configurada' if GEMINI_API_KEY else '❌ Não configurada'}")
print(f"   WaveSpeed:  {'✅ Configurada' if WAVESPEED_API_KEY else '❌ Não configurada'}")
print(f"   ngrok:      {'✅ Configurada' if NGROK_AUTH_TOKEN else '⚠️ Usando URL temporária'}")

print("\n✅ Arquivo .env criado com sucesso!")

## 🚀 Passo 4: Iniciar o Servidor

Execute esta célula para iniciar o ZERO STUDIO.

Um link público será gerado para acessar a interface web.

In [ ]:
import os
import threading
import time
from pyngrok import ngrok, conf

# Configurar ngrok se token fornecido
if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Garantir que estamos no diretório correto
os.chdir('/content/zero-studio')

# Função para executar o servidor Flask em background
def run_server():
    os.system('python web_server.py')

# Iniciar servidor em thread separada
server_thread = threading.Thread(target=run_server)
server_thread.daemon = True
server_thread.start()

# Aguardar servidor iniciar
print("⏳ Aguardando servidor iniciar...")
time.sleep(5)

# Criar túnel ngrok
try:
    public_url = ngrok.connect(5000)
    print("\n" + "="*60)
    print("🎬 ZERO STUDIO - Servidor Iniciado!")
    print("="*60)
    print(f"\n🌐 Acesse sua aplicação em:")
    print(f"\n   👉 {public_url}")
    print("\n" + "="*60)
    print("\n⚠️  Mantenha esta célula em execução!")
    print("    Fechar esta célula irá desligar o servidor.")
    print("\n💡 Dica: Você pode configurar as API keys pela interface web")
    print("    clicando no botão 'Configurar API Keys' no canto superior.")
except Exception as e:
    print(f"❌ Erro ao criar túnel: {e}")
    print("\nTentando alternativa com localtunnel...")
    !npm install -g localtunnel > /dev/null 2>&1
    !lt --port 5000

## 📊 Passo 5: Monitorar Logs (Opcional)

Execute esta célula para ver os logs do servidor em tempo real.

In [ ]:
# Monitorar arquivos de saída
import os
from IPython.display import clear_output
import time

output_dir = '/content/zero-studio/temp/outputs'

print("📊 Monitorando vídeos gerados...")
print("   (Execute novamente para atualizar)\n")

if os.path.exists(output_dir):
    videos = [f for f in os.listdir(output_dir) if f.endswith('.mp4')]
    if videos:
        print(f"🎥 Vídeos gerados ({len(videos)}):")
        for video in sorted(videos, key=lambda x: os.path.getmtime(os.path.join(output_dir, x)), reverse=True)[:10]:
            path = os.path.join(output_dir, video)
            size = os.path.getsize(path) / (1024*1024)  # MB
            print(f"   • {video} ({size:.2f} MB)")
    else:
        print("   Nenhum vídeo gerado ainda.")
else:
    print("   Pasta de outputs ainda não existe.")

## 📥 Passo 6: Baixar Vídeos Gerados (Opcional)

Execute esta célula para baixar os vídeos gerados para seu computador.

In [ ]:
from google.colab import files
import os
import zipfile

output_dir = '/content/zero-studio/temp/outputs'

if os.path.exists(output_dir):
    videos = [f for f in os.listdir(output_dir) if f.endswith('.mp4')]
    
    if videos:
        # Criar arquivo zip com todos os vídeos
        zip_path = '/content/zero_studio_videos.zip'
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for video in videos:
                video_path = os.path.join(output_dir, video)
                zipf.write(video_path, video)
        
        print(f"📦 {len(videos)} vídeo(s) compactado(s)!")
        print("⬇️  Iniciando download...\n")
        files.download(zip_path)
    else:
        print("❌ Nenhum vídeo encontrado para download.")
else:
    print("❌ Pasta de outputs não encontrada.")

## 🛑 Passo 7: Parar o Servidor (Opcional)

Execute esta célula quando quiser encerrar o servidor.

In [ ]:
from pyngrok import ngrok

# Desconectar todos os túneis ngrok
ngrok.kill()

print("🛑 Servidor encerrado!")
print("\n💡 Para reiniciar, execute novamente o Passo 4.")

---

## ❓ Ajuda e Solução de Problemas

### Erros Comuns:

**1. "Nenhuma voz disponível"**
- Verifique se a API key do ElevenLabs ou MiniMax está correta
- Configure as keys pelo botão na interface ou execute o Passo 3 novamente

**2. "Erro ao gerar vídeo"**
- Verifique se a API key do WaveSpeed está configurada
- Certifique-se de ter selecionado uma imagem/avatar

**3. "URL não funciona"**
- O ngrok pode ter expirado, execute o Passo 4 novamente
- Se continuar com problemas, adicione seu token do ngrok (gratuito em ngrok.com)

**4. "Servidor caiu"**
- O Colab pode ter desconectado por inatividade
- Execute os passos 2, 3 e 4 novamente

---

### Links Úteis:
- 🔑 [Obter API Key ElevenLabs](https://elevenlabs.io/)
- 🔑 [Obter API Key WaveSpeed](https://wavespeed.ai/)
- 🔑 [Obter Token ngrok (gratuito)](https://ngrok.com/)

---

**Desenvolvido com ❤️ usando ZERO STUDIO**